In [3]:
import pandas as pd

# Set the working directory
# observations_path = 'SWATPlusData/Zhangjiashan_data/observations/'
# observations_path = '../SWATPlusData/Tangnaihai_data/Observations/'
observations_path = '../SWATPlusData/Xunhua_data/Observations/'

日流量数据提取

In [4]:
# Read the streamflow data
# df = pd.read_csv('D:/DataSpace/HydroMeteAnthropicDatabase/3.RevisedHydroObsInfo/DailyStreamflow/JingRiver/41201100-ZhangJiaShan.csv',index_col=['TM'],parse_dates=['TM'])
df = pd.read_csv('D:/DataSpace/HydroMeteAnthropicDatabase/3.RevisedHydroObsInfo/DailyStreamflow/YellowRiver(黄河)/40100350-TangNaiHai.csv',index_col=['TM'],parse_dates=['TM'])

# Set the start year and the end year
start_year = 2001
end_year = 2019

cal_start = 2004
cal_end = 2011
val_start = 2012
val_end = 2019

# Select the data between the start year and the end year
df_day = df[str(start_year):str(end_year)]
# Write daily data to SWAT+ format
df_daily = pd.DataFrame()
df_daily['Date'] = df_day.index
df_daily['Flow'] = df_day['DailyFlow(m^3/s)'].values

# df_daily.to_csv(observations_path+'ZhangJiaShan_daily.csv',index=False)
df_daily.to_csv(observations_path+'TangNaiHai_daily.csv',index=False)

df_d = df[str(cal_start):str(cal_end)]
df_D = pd.DataFrame()
df_D['Index'] = [i+1 for i in range(len(df_d))]
df_D['Date'] = [str(date.day)+'/'+str(date.month)+'/'+str(date.year) for date in df_d.index]
df_D['Flow'] = df_d['DailyFlow(m^3/s)'].values
with open(observations_path+'TangNaiHai_daily-cal.txt','w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_D)):
        f.write(str(df_D['Index'][i])+'\t'+df_D['Date'][i]+'\t'+str(df_D['Flow'][i])+'\n')


df_d = df[str(val_start):str(val_end)]
df_D = pd.DataFrame()
df_D['Index'] = [i+1 for i in range(len(df_d))]
df_D['Date'] = [str(date.day)+'/'+str(date.month)+'/'+str(date.year) for date in df_d.index]
df_D['Flow'] = df_d['DailyFlow(m^3/s)'].values
with open(observations_path+'TangNaiHai_daily-val.txt','w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_D)):
        f.write(str(df_D['Index'][i])+'\t'+df_D['Date'][i]+'\t'+str(df_D['Flow'][i])+'\n')

月流量数据提取

In [28]:
Station = 'TangNaiHai'
df = pd.read_csv('D:/DataSpace/HydroMeteAnthropicDatabase/3.RevisedHydroObsInfo/MonthlyStreamflow/YellowRiver/40100350-{}.csv'.format(Station))
cal_start = 1969+3 # three warm up years
cal_end = 2011
val_strat = 2012
val_end = 2019
df_month = df[df['YEAR']>=cal_start]
new_df = pd.DataFrame()
new_df['Date'] = [str(i)+'/'+str(j).zfill(2) for i,j in zip(df_month['YEAR'].values,df_month['MONTH'].values)]
new_df['Flow'] = df_month['MonthlyFlow(m^3/s)'].values
# print(new_df)
new_df.to_csv(observations_path+'{}.csv'.format(Station),index=False)


# Select the df data between the year 2001 and 2010 based on the YEAR column
df_m = df[(df['YEAR']>=cal_start) & (df['YEAR']<=cal_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m['YEAR'].values,df_m['MONTH'].values)]
df_M['Flow'] = df_m['MonthlyFlow(m^3/s)'].values
with open(observations_path+'{}.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


df_m = df[(df['YEAR']>=val_strat) & (df['YEAR']<=val_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m['YEAR'].values,df_m['MONTH'].values)]
df_M['Flow'] = df_m['MonthlyFlow(m^3/s)'].values
with open(observations_path+'{}.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


#!年平均流量提取
# 将 year 和 month 列合并为 datetime 类型
df_yr = df[(df['YEAR']>=cal_start) & (df['YEAR']<=cal_end)]
df_yr['DATE'] = pd.to_datetime(df_yr[['YEAR', 'MONTH']].assign(DAY=1))
df_yr.set_index('DATE', inplace=True)
df_yr = df_yr.resample('YE').mean()
df_yr.rename(columns={'MonthlyFlow(m^3/s)': 'Flow'}, inplace=True)
print(df_yr)
df_Y = pd.DataFrame()
df_Y['Index'] = [i+1 for i in range(len(df_yr))]
df_Y['Date'] = [str(date.year) for date in df_yr.index]
df_Y['Flow'] = df_yr['Flow'].values
with open(observations_path+'{}_annual-cal.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_Y)):
        f.write(str(df_Y['Index'][i])+'\t'+df_Y['Date'][i]+'\t'+str(df_Y['Flow'][i])+'\n')

#!多年平均流量提取
# compute the avgerage of the annual flow
annual_avg_flow = df_yr['Flow'].mean()
print('1   long_term_avg   ',annual_avg_flow)
with open(observations_path+"{}_annual_avg-cal.txt".format(Station),"w") as f:
    f.write('Index,Date,Flow\n')
    f.write('1\tlong_term_avg\t'+str(annual_avg_flow))

                  STCD    YEAR  MONTH         Flow
DATE                                              
1972-12-31  40100350.0  1972.0    6.5   636.167665
1973-12-31  40100350.0  1973.0    6.5   565.659898
1974-12-31  40100350.0  1974.0    6.5   593.472427
1975-12-31  40100350.0  1975.0    6.5   976.292653
1976-12-31  40100350.0  1976.0    6.5   847.061471
1977-12-31  40100350.0  1977.0    6.5   516.288361
1978-12-31  40100350.0  1978.0    6.5   614.340207
1979-12-31  40100350.0  1979.0    6.5   648.269378
1980-12-31  40100350.0  1980.0    6.5   597.676520
1981-12-31  40100350.0  1981.0    6.5   929.581271
1982-12-31  40100350.0  1982.0    6.5   889.351318
1983-12-31  40100350.0  1983.0    6.5   990.365420
1984-12-31  40100350.0  1984.0    6.5   755.114136
1985-12-31  40100350.0  1985.0    6.5   698.473354
1986-12-31  40100350.0  1986.0    6.5   627.055317
1987-12-31  40100350.0  1987.0    6.5   557.907439
1988-12-31  40100350.0  1988.0    6.5   519.288060
1989-12-31  40100350.0  1989.0 

C:\Users\ZJY\AppData\Local\Temp\ipykernel_1840\1622885643.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yr['DATE'] = pd.to_datetime(df_yr.loc[:,['YEAR', 'MONTH']].assign(DAY=1))


In [11]:
Station = 'Xunhua'
df = pd.read_csv(f'D:/ResearchSpace/DataDrivenProcessDrivenFusionFramework/data/{Station.lower()}_natural_monthly_flow.csv',parse_dates=['date'],index_col=['date'])
cal_start = 1969+3 # three warm up years
cal_end = 2009
val_strat = 2010
val_end = 2014
df_month = df[df.index.year>=cal_start]
new_df = pd.DataFrame()
new_df['Date'] = [str(i)+'/'+str(j).zfill(2)+'/01' for i,j in zip(df_month.index.year.values,df_month.index.month.values)]
new_df['Flow'] = df_month['flow(m^3/s)'].values
# print(new_df)
new_df.to_csv(observations_path+'{}.csv'.format(Station),index=False)


# Select the df data between the year 2001 and 2010 based on the YEAR column
df_m = df[(df.index.year>=cal_start) & (df.index.year<=cal_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m.index.year.values,df_m.index.month.values)]
df_M['Flow'] = df_m['flow(m^3/s)'].values
with open(observations_path+'{}_monthly-cal.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


df_m = df[(df.index.year>=val_strat) & (df.index.year<=val_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m.index.year.values,df_m.index.month.values)]
df_M['Flow'] = df_m['flow(m^3/s)'].values
with open(observations_path+'{}_monthly-val.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


#!年平均流量提取
# 将 year 和 month 列合并为 datetime 类型
df_yr = df[(df.index.year>=cal_start) & (df.index.year<=cal_end)]
df_yr = df_yr.resample('YE').mean()
df_yr.rename(columns={'flow(m^3/s)': 'Flow'}, inplace=True)
print(df_yr)
df_Y = pd.DataFrame()
df_Y['Index'] = [i+1 for i in range(len(df_yr))]
df_Y['Date'] = [str(date.year) for date in df_yr.index]
df_Y['Flow'] = df_yr['Flow'].values
with open(observations_path+'{}_annual-cal.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_Y)):
        f.write(str(df_Y['Index'][i])+'\t'+df_Y['Date'][i]+'\t'+str(df_Y['Flow'][i])+'\n')

#!多年平均流量提取
# compute the avgerage of the annual flow
annual_avg_flow = df_yr['Flow'].mean()
print('1   long_term_avg   ',annual_avg_flow)
with open(observations_path+"{}_annual_avg-cal.txt".format(Station),"w") as f:
    f.write('Index,Date,Flow\n')
    f.write('1\tlong_term_avg\t'+str(annual_avg_flow))

                   Flow
date                   
1972-12-31   673.154614
1973-12-31   603.406455
1974-12-31   626.267274
1975-12-31  1001.673833
1976-12-31   911.451503
1977-12-31   558.346238
1978-12-31   662.402283
1979-12-31   690.041052
1980-12-31   638.657447
1981-12-31   962.032311
1982-12-31   891.410495
1983-12-31  1018.233834
1984-12-31   761.342718
1985-12-31   721.087268
1986-12-31   647.515060
1987-12-31   583.422447
1988-12-31   551.609643
1989-12-31  1058.569106
1990-12-31   541.041547
1991-12-31   489.540820
1992-12-31   668.983537
1993-12-31   719.811096
1994-12-31   553.364540
1995-12-31   523.427448
1996-12-31   480.825447
1997-12-31   480.269282
1998-12-31   611.407057
1999-12-31   799.936256
2000-12-31   493.240411
2001-12-31   474.387040
2002-12-31   368.611970
2003-12-31   572.418276
2004-12-31   523.676810
2005-12-31   848.094775
2006-12-31   487.416177
2007-12-31   626.642670
2008-12-31   583.906212
2009-12-31   865.659149
1   long_term_avg    665.0864237226863


In [12]:
Station = 'Guide'
df = pd.read_csv(f'D:/ResearchSpace/DataDrivenProcessDrivenFusionFramework/data/{Station.lower()}_natural_monthly_flow.csv',parse_dates=['date'],index_col=['date'])
cal_start = 1969+3 # three warm up years
cal_end = 2009
val_strat = 2010
val_end = 2014
df_month = df[df.index.year>=cal_start]
new_df = pd.DataFrame()
new_df['Date'] = [str(i)+'/'+str(j).zfill(2)+'/01' for i,j in zip(df_month.index.year.values,df_month.index.month.values)]
new_df['Flow'] = df_month['flow(m^3/s)'].values
# print(new_df)
new_df.to_csv(observations_path+'{}_monthly.csv'.format(Station),index=False)


# Select the df data between the year 2001 and 2010 based on the YEAR column
df_m = df[(df.index.year>=cal_start) & (df.index.year<=cal_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m.index.year.values,df_m.index.month.values)]
df_M['Flow'] = df_m['flow(m^3/s)'].values
with open(observations_path+'{}_monthly-cal.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


df_m = df[(df.index.year>=val_strat) & (df.index.year<=val_end)]
df_M = pd.DataFrame()
df_M['Index'] = [i+1 for i in range(len(df_m))]
df_M['Date'] =  [str(year)+'/'+str(month) for year,month in zip(df_m.index.year.values,df_m.index.month.values)]
df_M['Flow'] = df_m['flow(m^3/s)'].values
with open(observations_path+'{}_monthly-val.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_M)):
        f.write(str(df_M['Index'].values[i])+'\t'+df_M['Date'].values[i]+'\t'+str(df_M['Flow'].values[i])+'\n')


#!年平均流量提取
# 将 year 和 month 列合并为 datetime 类型
df_yr = df[(df.index.year>=cal_start) & (df.index.year<=cal_end)]
df_yr = df_yr.resample('YE').mean()
df_yr.rename(columns={'flow(m^3/s)': 'Flow'}, inplace=True)
print(df_yr)
df_Y = pd.DataFrame()
df_Y['Index'] = [i+1 for i in range(len(df_yr))]
df_Y['Date'] = [str(date.year) for date in df_yr.index]
df_Y['Flow'] = df_yr['Flow'].values
with open(observations_path+'{}_annual-cal.txt'.format(Station),'w') as f:
    f.write('Index,Date,Flow\n')
    for i in range(len(df_Y)):
        f.write(str(df_Y['Index'][i])+'\t'+df_Y['Date'][i]+'\t'+str(df_Y['Flow'][i])+'\n')

#!多年平均流量提取
# compute the avgerage of the annual flow
annual_avg_flow = df_yr['Flow'].mean()
print('1   long_term_avg   ',annual_avg_flow)
with open(observations_path+"{}_annual_avg-cal.txt".format(Station),"w") as f:
    f.write('Index,Date,Flow\n')
    f.write('1\tlong_term_avg\t'+str(annual_avg_flow))

                   Flow
date                   
1972-12-31   660.801722
1973-12-31   595.460850
1974-12-31   612.140466
1975-12-31   973.566079
1976-12-31   862.895573
1977-12-31   531.775463
1978-12-31   620.177873
1979-12-31   669.899096
1980-12-31   619.013760
1981-12-31   956.013118
1982-12-31   890.134154
1983-12-31  1010.959553
1984-12-31   781.531162
1985-12-31   727.006765
1986-12-31   645.881185
1987-12-31   586.769535
1988-12-31   549.582296
1989-12-31  1026.899759
1990-12-31   565.938171
1991-12-31   496.390606
1992-12-31   658.500645
1993-12-31   718.570778
1994-12-31   547.078810
1995-12-31   524.751634
1996-12-31   479.561935
1997-12-31   465.636141
1998-12-31   614.541526
1999-12-31   793.708780
2000-12-31   522.809722
2001-12-31   473.132662
2002-12-31   362.755599
2003-12-31   572.876956
2004-12-31   508.806089
2005-12-31   828.203876
2006-12-31   479.337039
2007-12-31   633.129052
2008-12-31   588.499510
2009-12-31   854.458441
1   long_term_avg    658.1367468895207
